# Laboratorio 6

Indicaciones generales:

* En lo que respecta a `C` no deberá usar punteros, ni arreglos. Tampoco se deberá emplear administración de memoria dinámica, ni de funciones auxiliares de ninguna librería.

* Los puntajes de ítems relacionados con mediciones de tiempo, error relativo y presentación de gráficos solo serán asignados en caso sus funciones estén correctamente implementadas.

* Las funciones implementadas solo deberán tener los argumentos mencionados en sus respectivos ítems.

* Los resultados de la función `_slow_` serán las referencias para medir los speedups.

* No se puede emplear ChatGPT, ni ningún modelo de lenguaje.

* Puede emplear apuntes, clases y hacer consultas en foros de internet.

In [1]:
# no borrar esta celda
! rm *.c *.o *.so

## Ejercicio

El coseno de un ángulo `x` se calcula con la siguiente serie:

$$
cos(x) = 1 - \frac{x^{2}}{2!} + \frac{x^{4}}{4!} - \frac{x^{6}}{6!} + \frac{x^{8}}{8!} - \cdots
$$

Durante el laboratorio se le solicitará implementar una grupo de funciones basados en esta serie.

1. Implementar una función en `python` que calcule el coseno de un ángulo `x`.

Nombre: `py_slow_cos`

Argumentos:
* `x` : ángulo
* `n_terms`: cantidad de términos

Resultado: `cos(x)`

Observaciones: 

* No debe emplear funciones de módulos auxiliaries, ni suyas propias. 
* Todo se debe hacer dentro de la función.

In [11]:
def py_slow_cos(x,n_terms):
    cos_x = 0
    for n in range(n_terms):
        d = 1
        for i in range(1,2*n+1):
            d = d*i
        tn = (((-1)**n))*(x**(2*n))/(1.0*d)
        cos_x += tn
    return cos_x

2. Implementar una función en `python` que calcule el coseno de un ángulo `x`.

Nombre: `py_fast_cos`

Argumentos:
* `x` : ángulo
* `n_terms`: cantidad de términos

Resultado: `cos(x)`

Observaciones: 
* No puede emplear factorial de ninguna manera. 
* No debe emplear funciones de módulos auxiliares, ni suyas propias.
* Debe ser iterativo.
* Sugerencia: Deducir el término $t_{n}$ en función del término $t_{n-1}$

In [12]:
def py_fast_cos(x,n_terms):
    cos_x = 0
    tn = 1
    for n in range(1,n_terms+1):
        cos_x += tn
        tn = (-1.0*x**2/((2*n)*(2*n-1)))*tn
    return cos_x

3. En una celda mágica implementar dos versiones en `C` para la función `py_fast_cos`. Una versión trabajará con tipo `double` y otra trabajará con tipo `long double`. Considere que estas funciones tienen los mismos argumentos que la función `py_fast_cos`.

Observaciones:

* Justifique el tipo de dato de cada uno de los argumentos.
* Justifique el tipo de dato que devuelve la función.

In [13]:
%%file funcs.c
#include <stdint.h>
double c_d_cos(double x, uint64_t n_terms){
    double cos_x = 0.0;
    double tn = 1.0;
    double den;
    for(uint64_t n=1;n<=n_terms;n++){
        cos_x = cos_x + tn;
        den = (double)((2*n)*(2*n-1));
        tn = -1.0*(x*x)*tn/den;
    }
    return cos_x;
}

long double c_ld_cos(long double x, uint64_t n_terms){
    long double cos_x = 0.0;
    long double tn = 1.0;
    long double den;
    for(uint64_t n=1;n<=n_terms;n++){
        cos_x = cos_x + tn;
        den = (long double)((2*n)*(2*n-1));
        tn = -1.0*(x*x)*tn/den;
    }
    return cos_x;
}

Overwriting funcs.c


4. Crear el *object file* y la *shared library*

In [14]:
! gcc -c funcs.c -o funcs.o

In [15]:
! gcc -fPIC -shared funcs.o -o funcs.so

5. Hacer una función que enlace con `Python` mediante `ctypes` las funciones anteriores. Esta función debe devolver las dos funciones configuradas.

In [16]:
import ctypes
import numpy as np

def ctypes_funcs():
    lib = ctypes.CDLL('./funcs.so')

    lib.c_d_cos.argtypes = [ctypes.c_double, ctypes.c_uint64]
    lib.c_d_cos.restype = ctypes.c_double

    lib.c_ld_cos.argtypes = [ctypes.c_longdouble, ctypes.c_uint64]
    lib.c_ld_cos.restype = ctypes.c_longdouble

    return lib.c_d_cos, lib.c_ld_cos

6. Haga una instancia de las dos funciones

In [17]:
c_d_cos, c_ld_cos = ctypes_funcs()

7. Implemente una función que de forma iterativa encuentre la cantidad de términos que requieren sus funciones para calcular con una determinada precisión el seno de un grupo de ángulos definidos entre $[inf, sup]$.

Nombre de la función: `encontrar_n_terms`

Argumentos:

* `f`: función

* `inf`: extremo izquierdo del dominio

* `sup`: extremo derecho del dominio

* `n_angs`: cantidad de ángulos

* `pre`: precisión

Resultado: Cantidad de términos

Observaciones: 
* Puede usar la función `linspace` del módulo `numpy` para crear su vector de ángulos, y la función `norm` del submódulo `linalg` del módulo `numpy` para calcular la norma de un arreglo. 
* Para su referencia considere el resultado de la función `cos` del módulo `numpy`.



In [24]:
def encontrar_n_terms(f,inf,sup,n_angs,pre):
    ang = np.linspace(inf,sup,num=n_angs)
    n_t = 1
    err = 0
    while(True):
        ref = np.cos(ang)
        ref = np.array(ref).astype(np.double)
        res = f(ang,n_t)
        res = np.array(res).astype(np.double)
        err = np.linalg.norm(res-ref)/np.linalg.norm(ref)
        if (err<pre):
            break
        n_t+=1
    return n_t

8. Haga una prueba de `encontrar_n_terms` para cada una de sus funciones. 

Considere:
* `inf` = -2pi
* `sup` = 2pi
* `n_angs` = 1000
* `pre` = 4e-15

Observación: Cada resultado debe estar asignado a una variable independiente y ser mostrado en una celda

In [29]:
inf = -2*np.pi
sup = 2*np.pi
n_angs = 1000
pre = 4*1e-15
fD = c_d_cos
n_termsS = encontrar_n_terms(py_slow_cos,inf,sup,n_angs,pre)
print(n_termsS)
n_termsF = encontrar_n_terms(py_fast_cos,inf,sup,n_angs,pre)
print(n_termsF)
n_termsD = encontrar_n_terms(c_d_cos,inf,sup,n_angs,pre)
print(n_termsD)
n_termsLD = encontrar_n_terms(c_ld_cos,inf,sup,n_angs,pre)
print(n_termsLD)

ArgumentError: argument 1: <class 'TypeError'>: wrong type

9. Implemente una función que realice una cantidad de mediciones de tiempo de alguna de sus funciones que calcula el coseno y devuelva la mediana de esas mediciones.

Nombre de la función: `encontrar_mediana_de_mediciones_cos`
Argumentos:
* `f`: función
* `ang`: ángulo de la función
* `n_terms`: cantidad de términos de la función
* `n_iter`: cantidad de iteraciones

Resultado: Mediana de las mediciones realizadas.

Observación: Puede emplear la función `median` del módulo `statistics`.

In [26]:
import time
import statistics
import matplotlib.pyplot as plt

def encontrar_mediana_de_mediciones_cos(f,ang,n_terms,n_iter):
    lst= []
    for i in range(n_iter):
        tic = time.perf_counter()
        f(ang,n_terms)
        toc = time.perf_counter()
        lst.append(toc-tic)
    return statistics.median(lst)

10. Haga dos pruebas de `encontrar_mediana_de_mediciones_cos` para cada una de sus funciones. 

Prueba 1 :
* `ang` = `inf`
* `n_iter` = 50

Prueba 2 :
* `ang` = `sup`
* `n_iter` = 50

Observaciones:

* Cada función empleará su cantidad de términos previamente calculados
* Cada resultado deberá ser asignado a una variable independiente

In [27]:
#prueba1
ang = inf
n_iter = 50
n_terms = 19
mediana1_s = encontrar_mediana_de_mediciones_cos(py_slow_cos,ang,n_terms,n_iter)
mediana1_f = encontrar_mediana_de_mediciones_cos(py_fast_cos,ang,n_terms,n_iter)
mediana1_d = encontrar_mediana_de_mediciones_cos(c_d_cos,ang,n_terms,n_iter)
mediana1_ld = encontrar_mediana_de_mediciones_cos(c_ld_cos,ang,n_terms,n_iter)
print(mediana1_s)
print(mediana1_f)
print(mediana1_d)
print(mediana1_ld)

1.6674997823429294e-05
2.9255024855956435e-06
5.119982233736664e-07
5.129986675456166e-07


In [ ]:
#prueba2
ang = sup
n_iter = 50
n_terms = 19
mediana2_s = encontrar_mediana_de_mediciones_cos(py_slow_cos,ang,n_terms,n_iter)
mediana2_f = encontrar_mediana_de_mediciones_cos(py_fast_cos,ang,n_terms,n_iter)
mediana2_d = encontrar_mediana_de_mediciones_cos(c_d_cos,ang,n_terms,n_iter)
mediana2_ld = encontrar_mediana_de_mediciones_cos(c_ld_cos,ang,n_terms,n_iter)
print(mediana2_s)
print(mediana2_f)
print(mediana2_d)
print(mediana2_ld)

11. Presente gráficos de barras de las medianas de los tiempos y de los speedups a partir de sus resultados del ítem anterior

In [ ]:
ang1 = inf
ang2 = sup
medianas1 = [mediana1_s,mediana1_f,mediana1_d,mediana1_ld]
medianas2 = [mediana2_s,mediana2_f,mediana2_d,mediana2_ld]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
casos = ['Python lento', 'Python rapido', 'C double', 'C long double']
ax.bar(casos, medianas1)
plt.title(f'Mediana de tiempos de cos({ang1}) con {n_terms} términos')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
casos = ['Python lento', 'Python rapido', 'C double', 'C long double']
ax.bar(casos, medianas2)
plt.title(f'Mediana de tiempos de cos({ang2}) con {n_terms} términos')
plt.show()

In [ ]:
speedups1 = [mediana1_s/mediana1_f, mediana1_s/mediana1_d, mediana1_s/mediana1_ld]
speedups2 = [mediana2_s/mediana2_f, mediana2_s/mediana2_d, mediana2_s/mediana2_ld]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
casos = ['Python rapido', 'C double', 'C long double']
ax.bar(casos, speedups1)
plt.title(f'Speedup de cos({ang1}) con {n_terms} términos')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
casos = ['Python rapido', 'C double', 'C long double']
ax.bar(casos, speedups1)
plt.title(f'Speedup de cos({ang2}) con {n_terms} términos')
plt.show()

12. Implemente una función en `Python` que calcule el seno de un arreglo de ángulos.

Nombre de la función: `calc_cosens`

Argumentos:
* `f`: función
* `inf`: extremo izquierdo del dominio
* `sup`: extremo derecho del dominio
* `n_angs`: cantidad de ángulos
* `n_terms`: cantidad de términos de la función `f`

Resultado: Arreglo de `numpy` con los senos de los ángulos

Observación: Puede emplear la función `linspace` del módulo `numpy` para crear su arreglo de ángulos.

In [ ]:
def calc_cosens(f,inf,sup,n_angs,n_terms):
    ang = np.linspace(inf,sup,num=n_angs)
    lt = []
    for i in range(n_angs):
        lt.append(f(ang[i],n_terms))
    return lt

13. Implemente una función que realice una cantidad de mediciones de tiempo de su función anterior y devuelva la mediana de esas mediciones.

Nombre de la función: `encontrar_mediana_de_mediciones_calc_cosens`

Argumentos:
* `f`: función
* `inf`: extremo izquierdo del dominio
* `sup`: extremo derecho del dominio
* `n_terms`: cantidad de términos de la función
* `n_iter`: cantidad de iteraciones

Resultado: Mediana de las mediciones realizadas.

Observación: Puede emplear la función `median` del módulo `statistics`.

In [ ]:
def encontrar_mediana_de_mediciones_calc_cosens(f,inf,sup,n_angs,n_terms,n_iter):
    lst = []
    for i in range(n_iter):
        tic = time.perf_counter()
        calc_cosens(f,inf,sup,n_angs,n_terms)
        toc = time.perf_counter()
        lst.append(toc-tic)
    return statistics.median(lst)

14. Haga pruebas de `encontrar_mediana_de_mediciones_cosens` para cada una de sus funciones. 

Observaciones:

* Los valores para `inf`, `sup`, `n_angs` y `n_iter` serán los mismos que los definidos anteriormente
* Cada función empleará su cantidad de términos previamente calculados
* Cada resultado deberá ser asignado a una variable independiente

In [ ]:
inf = -2*np.pi
sup = 2*np.pi
n_angs = 1000
n_terms = 19
n_iter = 50
result1 = encontrar_mediana_de_mediciones_calc_cosens(py_slow_cos,inf,sup,n_angs,n_terms,n_iter)
result2 = encontrar_mediana_de_mediciones_calc_cosens(py_fast_cos,inf,sup,n_angs,n_terms,n_iter)
result3 = encontrar_mediana_de_mediciones_calc_cosens(c_d_cos,inf,sup,n_angs,n_terms,n_iter)
result4 = encontrar_mediana_de_mediciones_calc_cosens(c_ld_cos,inf,sup,n_angs,n_terms,n_iter)
print(result1,result2,result3,result4)

15. Presente gráficas de barras de las medianas calculadas en el item anterior y de los speedups a partir de los resultados del ítem anterior.

In [ ]:
medianas = [result1,result2,result3,result4]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
casos = ['Python lento', 'Python rapido', 'C double', 'C long double']
ax.bar(casos, medianas)
plt.title(f'Mediana de mediciones')
plt.show()

In [ ]:
speedup = [result1/result2,result1/result3,result1/result4]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
casos = ['Python rapido', 'C double', 'C long double']
ax.bar(casos, speedup)
plt.title(f'Mediana de mediciones')
plt.show()

## Distribución de puntaje

| ítem | puntos |
|:----:|:------:|
|   1  |    1   |
|   2  |    2   |
|   3  |    4   |
|   4  |   0.5  |
|   5  |   0.5  |
|   6  |   0.5  |
|   7  |   1.5  |
|   8  |   0.5  |
|   9  |   1.5  |
|  10  |   0.5  |
|  11  |   0.5  |
|  12  |   0.5  |
|  13  |   0.5  |
|  14  |   0.5  |
|  15  |   0.5  |